In [116]:
import pandas as pd
import json
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Read the text file line by line
with open('json/train.txt', 'r') as file:
    lines = file.readlines()

# Parse each line as a JSON object
data = [json.loads(line.strip()) for line in lines]
data = data[:2]
# Convert the list of JSON objects to a pandas DataFrame
df = pd.DataFrame(data)


In [117]:
text = df.iloc[0]["text"]
edits = df.iloc[0]["edits"][0][1]

In [118]:
# modifiedRow = []
# for (id, row) in df.iterrows():
#     edits = row['edits'][0][1]
#     for i in range(len(edits)):
#         if(edits[i][2] != None):
#             text = row['text']
#             for j in range(i - 1, -1, -1):
#                 if(edits[j][2] != None):
#                     start = edits[j][0]
#                     end = edits[j][1]
#                     replacement = edits[j][2]
#                     text = text[:start] + replacement + text[end:]

#             newRow = {}
            # newRow['text'] = tokenizer.encode(text, add_special_tokens=True, max_length=600, pad_to_max_length=True, truncation=True)
#             newRow['error_position'] = [edits[i][0], edits[i][1]]
#             newRow['correction'] = tokenizer.encode(edits[i][2], add_special_tokens=True, max_length=5, pad_to_max_length=True, truncation=True)
#             modifiedRow.append(newRow)

In [119]:
def index_mapping(text):
    tokenized_text = tokenizer.tokenize(text, add_special_tokens=True)
    index_token = []
    i = 0
    for token in tokenized_text:
        token = token.lstrip('#')
        while(text[i : i + len(token)] != token):
            i += 1
        index_token.append([i, i + len(token)])        
        i += len(token)
    return index_token

def get_error_position(text, edits):
    index_token = index_mapping(text)
    error_position = []
    index_token_ind = 0
    for edit in edits:
        while(index_token_ind < len(index_token) and index_token[index_token_ind][0] < edit[0]):
            index_token_ind += 1
        start = index_token_ind
        while(index_token_ind < len(index_token) and index_token[index_token_ind][1] < edit[1]):
            index_token_ind += 1
        end = index_token_ind
        error_position.append([start, end, text[edit[0]: edit[1]], edit[2]])
    return error_position

def get_format_errors(text, edits):
    tokenized = tokenizer(text, add_special_tokens=True, max_length=640, pad_to_max_length=True, truncation=True, return_tensors="np")
    input_ids = tokenized['input_ids'][0]
    input_attention = tokenized['attention_mask'][0]
    error_position = get_error_position(text, edits)
    input_ids = input_ids[1:-1]
    formatted_errors = []
    i = 0
    error_ind = 0
    while(i < len(input_ids)):
        include_tokens = input_ids[i: i + 128]
        inlcude_attention = input_attention[i: i + 128]
        current_errors = []
        while(error_ind < len(error_position)):
            if(error_position[error_ind][0] >= i and error_position[error_ind][1] <= i + 128):
                current_errors.append(error_position[error_ind])
                current_errors[-1][0] -= i
                current_errors[-1][1] -= i
                error_ind += 1
            else: break
        while(len(include_tokens) < 128): 
            include_tokens = np.append(include_tokens, tokenizer.pad_token_id)
            inlcude_attention = np.append(inlcude_attention, 0)
        formatted_errors.append({"input": include_tokens, "attention": inlcude_attention, "edits": current_errors, "text": input_ids})
        i += 128
    return formatted_errors

In [120]:
len(get_format_errors(text, edits))

c:\Users\dk101\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Keyword arguments {'add_special_tokens': True} not recognized.


5

In [121]:
print(tokenizer.tokenize("Hi! I'm deepak", add_special_tokens=True))
print(tokenizer("Hi! I'm deepak", return_tensors="np", add_special_tokens=True))

Keyword arguments {'add_special_tokens': True} not recognized.


['Hi', '!', 'I', "'", 'm', 'deep', '##ak']
{'input_ids': array([[ 101, 8790,  106,  146,  112,  182, 1996, 3715,  102]]), 'token_type_ids': array([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [122]:
final_data = []
for _, row in df.iterrows():
    if(_%100 == 0): print(_)
    text = row['text']
    edits = row['edits'][0][1]
    formatted_errors = get_format_errors(text, edits)
    for formatted_error in formatted_errors:
        for i in range(len(formatted_error["edits"])):
            last_wrong = formatted_error["edits"][i - 1][1] if i > 0 else 0
            correction = tokenizer.encode(formatted_error["edits"][i][-1], add_special_tokens=True, pad_to_max_length=True, truncation=True)[1:-1]
            while(len(correction) < 5):
                correction.append(tokenizer.pad_token_id)
                
            final_data.append({
                "input": formatted_error["input"],
                "attention": formatted_error["attention"],
                "edits": formatted_error["edits"][i][:-2],
                "text": formatted_error["text"],
                "last_wrong": last_wrong,
                "correction": correction
            })

Keyword arguments {'add_special_tokens': True} not recognized.
Keyword arguments {'add_special_tokens': True} not recognized.


0


In [123]:
df = pd.DataFrame(final_data)

In [124]:
df

,input,attention,edits,text,last_wrong,correction
0,"[12956, 2203, 1137, 10779, 2312, 117, 146, 182...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[15, 15]","[12956, 2203, 1137, 10779, 2312, 117, 146, 182...",0,"[1114, 0, 0, 0, 0]"
1,"[12956, 2203, 1137, 10779, 2312, 117, 146, 182...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[26, 27]","[12956, 2203, 1137, 10779, 2312, 117, 146, 182...",15,"[1486, 0, 0, 0, 0]"
2,"[12956, 2203, 1137, 10779, 2312, 117, 146, 182...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[35, 35]","[12956, 2203, 1137, 10779, 2312, 117, 146, 182...",27,"[1113, 0, 0, 0, 0]"
3,"[12956, 2203, 1137, 10779, 2312, 117, 146, 182...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[63, 63]","[12956, 2203, 1137, 10779, 2312, 117, 146, 182...",35,"[3761, 0, 0, 0, 0]"
4,"[12956, 2203, 1137, 10779, 2312, 117, 146, 182...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[64, 64]","[12956, 2203, 1137, 10779, 2312, 117, 146, 182...",63,"[1104, 0, 0, 0, 0]"
5,"[12956, 2203, 1137, 10779, 2312, 117, 146, 182...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[72, 72]","[12956, 2203, 1137, 10779, 2312, 117, 146, 182...",64,"[1272, 1104, 0, 0, 0]"
6,"[12956, 2203, 1137, 10779, 2312, 117, 146, 182...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[83, 83]","[12956, 2203, 1137, 10779, 2312, 117, 146, 182...",72,"[1120, 0, 0, 0, 0]"
7,"[1103, 4997, 1104, 1155, 1108, 1115, 5595, 105...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[15, 18]","[12956, 2203, 1137, 10779, 2312, 117, 146, 182...",0,"[1130, 1901, 0, 0, 0]"
8,"[1103, 4997, 1104, 1155, 1108, 1115, 5595, 105...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[25, 25]","[12956, 2203, 1137, 10779, 2312, 117, 146, 182...",18,"[1111, 0, 0, 0, 0]"
9,"[1103, 4997, 1104, 1155, 1108, 1115, 5595, 105...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[39, 39]","[12956, 2203, 1137, 10779, 2312, 117, 146, 182...",25,"[7286, 0, 0, 0, 0]"


In [127]:
ids = np.array([np.array(i).astype(np.float32) for i in df['input']])
attention = np.array([np.array(len(i)).astype(np.float32) for i in df["attention"]])
text = np.array([np.array(i).astype(np.float32) for i in df["text"]])
last_wrong_position = np.array([i for i in df['last_wrong']]).astype(np.float32)
error_position = np.array([np.array(i).astype(np.float32) for i in df['edits']]).astype(np.float32)
correction = np.array([np.array(len(i)).astype(np.float32) for i in df['correction']]).astype(np.float32)

In [128]:
text.shape

(22, 638)

In [69]:
import tensorflow as tf
import numpy as np
import keras

In [78]:
inputText = tf.keras.layers.Input(shape=(128,), name='inputText')
reshaped_inputs = tf.keras.layers.Reshape((128, 1))(inputText)
lstm = tf.keras.layers.LSTM(128, return_sequences=True)(reshaped_inputs)
lstm = tf.keras.layers.LSTM(64)(lstm)
dense = tf.keras.layers.Dense(1024, activation='relu')(lstm)
dense = tf.keras.layers.Dense(512, activation='relu')(dense)
dense = tf.keras.layers.Dense(256, activation='relu')(dense)
dense = tf.keras.layers.Dense(512, activation='relu')(dense)
dense = tf.keras.layers.Flatten()(dense)

inputLastPosition = tf.keras.layers.Input(shape=(1,), name='last_wrong_position')
denseLastPosition = tf.keras.layers.Dense(64, activation='relu')(inputLastPosition)
denseLastPosition = tf.keras.layers.Dense(32, activation='relu')(denseLastPosition)
denseLastPosition = tf.keras.layers.Dense(16, activation='relu')(denseLastPosition)

concatenated = tf.keras.layers.concatenate([dense, denseLastPosition])

denseErrors = tf.keras.layers.Dense(1024, activation='relu')(concatenated)
denseErrors = tf.keras.layers.Dense(512, activation='relu')(denseErrors)
denseErrors = tf.keras.layers.Dense(256, activation='relu')(denseErrors)
denseErrors = tf.keras.layers.Dense(128, activation='relu')(denseErrors)
denseErrors = tf.keras.layers.Dense(64, activation='relu')(denseErrors)
outputErrors = tf.keras.layers.Dense(2, activation='relu', name="location")(denseErrors)

In [79]:
import tensorflow as tf

model = tf.keras.Model(inputs=[inputText, inputLastPosition], outputs=[outputErrors])
model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=["accuracy", keras.metrics.MeanSquaredError()])

ValueError: The name "inputText" is used 2 times in the model. All operation names should be unique.

In [ ]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputText           │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 128, 1)    │          0 │ inputText[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 128, 128)  │     66,560 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     49,408 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1024)      │     66,560 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 512)       │    524,800 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ last_wrong_position │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │    131,328 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │        128 │ last_wrong_posit… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 512)       │    131,584 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 32)        │      2,080 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 512)       │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 16)        │        528 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 528)       │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 1024)      │    541,696 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 512)       │    524,800 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 256)       │    131,328 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 128)       │     32,896 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 64)        │      8,256 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ location (Dense)    │ (None, 2)         │        130 │ dense_11[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,212,082 (8.44 MB)

 Trainable params: 2,212,082 (8.44 MB)

 Non-trainable params: 0 (0.00 B)

In [80]:
text = np.array([np.array(i).astype(np.float32) for i in df['input']])
last_wrong_position = np.array([i for i in df['last_wrong']]).astype(np.float32)
error_position = np.array([np.array(i).astype(np.float32) for i in df['edits']]).astype(np.float32)
correction = np.array([np.array(i).astype(np.float32) for i in df['correction']]).astype(np.float32)

In [ ]:
model.fit([text, last_wrong_position], [error_position], epochs=1000, batch_size=32)

In [83]:
model.predict([text[3:7], last_wrong_position[3:7]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step


array([[ 0.      , 61.731167],
       [ 0.      , 67.54851 ],
       [ 0.      , 69.1967  ],
       [ 0.      , 82.967896]], dtype=float32)

In [84]:
last_wrong_position[3:7], error_position[3:7]

(array([35., 63., 64., 72.], dtype=float32),
 array([[63., 63.],
        [64., 64.],
        [72., 72.],
        [83., 83.]], dtype=float32))